Смоделировать игру против лотерейного автомата типа “777”. Игрок платит 1 руб.,
после чего выпадает случайное целое число, равномерно распределенное от 0 до 999.
При некоторых значениях числа игрок получает выигрыш (см. справа)

Выгодна ли игра игроку?

Сколько в среднем приобретает
или теряет игрок за одну игру?

- 777: 200 руб.
- 999: 100 руб.
- 555: 50 руб.
- 333: 15 руб.
- 111: 10 руб.
- *77: 5 руб.
- **7: 3 руб.
- *00: 2 руб.
- **0: 1 руб.

* – любая цифра


**Дополнительное задание повышенной сложности.**

Теоретически рассчитать средний выигрыш (проигрыш) и сравнить с результатами моделирования

In [32]:
import random



In [31]:
from typing import Iterator
class Gambler:
    __winning_nums =  {
        777: 200,
        999: 100,
        555: 50,
        333: 15,
        111: 10
    }

    __winning_lookups = [
        (lambda s: s.endswith("77"), 5), # *77
        (lambda s: s.endswith("7"), 3), # **7
        (lambda s: s.endswith("00"), 2), # *00
        (lambda s: s.endswith("0"), 1), # **0
    ]

    def __init__(self) -> None:
        self._randomizer = random.SystemRandom()

    def __iter__(self) -> Iterator[int]:
        for _ in iter(int, 1):
            combination = self._randomizer.randint(0, 1000)
            if combination in self.__winning_nums:
                yield self.__winning_nums[combination]
                continue

            combination_str = str(combination).rjust(3, '0')
            for test_func, value in self.__winning_lookups:
                if test_func(combination_str):
                    yield value
                    continue

            yield 0

In [35]:
%%timeit
gambler = Gambler()
gambler_iter = iter(gambler)

iterations = 100000
gamer_won = 0

for snatch in range(0, iterations):
    gamer_won += next(gambler_iter)

print(
    f"Игр: {iterations} \n"
    f"Потрачено денег: {iterations} \n"
    f"Выиграно денег: {gamer_won}\n"
    f"Игрок приобретает в среднем за 1 игру: {(gamer_won - iterations) / iterations} денег"
)

# Игра не выгодна, при увеличении количества интераций средний проигрыш колеблется в диапозоне 0,3 - 0,32 рублей
# При итерациях меньше 1000 игра иногда может быть выгодной

Игр: 100000 
Потрачено денег: 100000 
Выиграно денег: 69353
Игрок приобретает в среднем за 1 игру: -0.30647 денег
Игр: 100000 
Потрачено денег: 100000 
Выиграно денег: 69909
Игрок приобретает в среднем за 1 игру: -0.30091 денег
Игр: 100000 
Потрачено денег: 100000 
Выиграно денег: 68327
Игрок приобретает в среднем за 1 игру: -0.31673 денег
Игр: 100000 
Потрачено денег: 100000 
Выиграно денег: 69282
Игрок приобретает в среднем за 1 игру: -0.30718 денег
Игр: 100000 
Потрачено денег: 100000 
Выиграно денег: 68453
Игрок приобретает в среднем за 1 игру: -0.31547 денег
Игр: 100000 
Потрачено денег: 100000 
Выиграно денег: 69520
Игрок приобретает в среднем за 1 игру: -0.3048 денег
Игр: 100000 
Потрачено денег: 100000 
Выиграно денег: 67296
Игрок приобретает в среднем за 1 игру: -0.32704 денег
Игр: 100000 
Потрачено денег: 100000 
Выиграно денег: 69499
Игрок приобретает в среднем за 1 игру: -0.30501 денег
1.35 s ± 105 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Теоретический расчет среднего выигрыша

События:

- P(выигрыша) = количество всех выигрышных комбинаций/ все возможные комбинации
- P(проигрыша) = 1 - P(выигрыша)

- P(любая цифра) = 1 или 100%

- P(конкретная цифра) = 0,1 - 10%, один шанс из 10
- P(все кроме конкретной цифры) = 1- P(конкретная цифра) = 1 - 0,1

- P(&lowast;&lowast;7) = 1&lowast;1&lowast;0,1
- P(&lowast;77) =  = 1&lowast;0,1&lowast;0,1
- P(777)= P(конкретная цифра) &lowast; P(конкретная цифра) &lowast; P(конкретная цифра) = 0,1&lowast;0,1&lowast;0,1

In [38]:
prob_777 = 0.1*0.1*0.1 #вероятность получения 200р
prob_999 = 0.1*0.1*0.1 #вероятность получения 100р
prob_555 = 0.1*0.1*0.1 #вероятность получения 50р
prob_333 = 0.1*0.1*0.1 #вероятность получения 15р
prob_111 = 0.1*0.1*0.1 #вероятность получения 10р
prob_x77 = (1*0.1*0.1) - prob_777 #вероятность получения 5р
prob_xx7 = (1*1*0.1) - prob_777 - prob_x77 #вероятность получения 3р
prob_x00 = 1*0.1*0.1 #вероятность получения 2р
prob_xx0 = (1*1*0.1) - prob_x00 #вероятность получения 1р

prob_winning = (prob_777+ prob_999+ prob_555 + prob_333 +
                prob_111 + prob_x77 + prob_xx7 + prob_x00 + prob_xx0) # вероятность получения выигрыша

average_winning = (
    (prob_777*200)+
    (prob_999*100)+
    (prob_555*50)+
    (prob_333*15)+
    (prob_111*10)+
    (prob_x77*5)+
    (prob_xx7*3)+
    (prob_x00*2)+
    (prob_xx0*1) )

average_losing = -1


print(f"Теоретический чистый средний выигрыш: {average_winning} \n"
      f"Теоретический чистый средний проигрыш: {average_losing} \n"
      f"Выгода: {round(average_winning+average_losing, 3)}")

Теоретический чистый средний выигрыш: 0.8 
Теоретический чистый средний проигрыш: -1 
Выгода: -0.2


Теоретический расчет среднего выигрыша имеет близкое значение с результатами моделирования.
Хотя, погрешность достаточно большая